# The optimised computation library of RooFit
Author: Stephan Hageboeck

Here, we test the impact of using a computation library that is optimised for different CPU models.
When RooFit is loaded for the first time, the CPU capabilities are inspected, and a computation library targeted for that CPU is loaded.

We visualise this with a little hack here:

In [1]:
gDebug = 1;

With ROOT's `gDebug` set to 1, ROOT prints things that happen behind the scenes.Let's now use a RooFit class, which triggers the loading of the RooFit libraries.

Note that ROOT loads a library with a suffix such as `_AVX2.so` (depending on the CPU where this runs). 

We turn `gDebug` off, and now go on to run some fits.

In [2]:
RooRealVar __var;
gDebug = 0;


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

In roofitcore/InitUtils.cxx:loadComputeLibrary(): Library libRooBatchCompute_AVX2 was loaded successfully


Info in <TUnixSystem::Load>: loaded library /cvmfs/sft-nightlies.cern.ch/lcg/views/dev3/Mon/x86_64-centos7-gcc8-opt/lib/libRooBatchCompute_AVX2.so, status 0
Info in <TUnixSystem::Load>: loaded library /cvmfs/sft-nightlies.cern.ch/lcg/nightlies/dev3/Mon/ROOT/HEAD/x86_64-centos7-gcc8-opt/lib/libRooFitCore.so, status 1


# Let's run a speed test
Let's test a fit of the form
```
Model(x, <parameters>) = frac * Gauss(x | mean, sigma) + (1 - frac) * Bernstein_5(x | a1, ..., a5)
```

### Step 1: Create a background model (Bernstein polynomials)

In [3]:
RooRealVar  x("x", "x", -100, 50);
RooRealVar a1("a1", "a1", 0.8, 0.6, 1.2);
RooRealVar a2("a2", "a2", 0.0, -1.0, 1.0);
RooRealVar a3("a3", "a3", 0.09, 0.05, 0.4);
RooRealVar a4("a4", "a4", 0.0, 0.2, 0.8);
RooRealVar a5("a5", "a5", 0.09, 0.05, 0.5);
a4.setConstant();
a3.setConstant();
a2.setConstant();

RooBernstein bernstein("bernstein", "bernstein PDF 5 coefficients", x, RooArgSet(a1, a2, a3, a4, a5));

### Step 2: Create a signal model (Gaussian)

In [4]:
RooRealVar mean("mean", "mean", 10, -100, 50);
RooRealVar sigma("sigma", "sigma", 5, -10, 50);
RooGaussian gauss("gauss", "Gaussian signal model", x, mean, sigma);

[#0] WARNING:InputArguments -- The parameter 'sigma' with range [-10, 50] of the RooGaussian 'gauss' exceeds the safe range of (0, inf). Advise to limit its range.


### Step 3: Create a sum model

In [5]:
RooRealVar fracSig("sigFrac", "Number of signal events", 0.1, 0, 1);
RooAddPdf model("model", "Gauss signal + Bernstein background", RooArgSet(gauss, bernstein), RooArgSet(fracSig));

We save the initial parameters of the model, so we can reuse them later

In [6]:
RooArgSet parameters(a1, a2, a3, a4, a5, mean, sigma, fracSig);
RooArgSet* initialParameters = parameters.snapshot();

# Generate data and fit model
We create a dataset by sampling events from the model

In [7]:
auto dataSet = model.generate(x, 1000000);

# Let's run some fits

### 1. Run fit with classic RooFit computation
We first load the initial parameters (so that all fits start from the same point), and then we fit the model with 2 cores (SWAN doesn't provide more).
Note that the fitter often chooses a parameter point with negative sigma, where the model is undefined. It prints a message, but it recovers and finishes the fit.

After the fit, we print the parameters and the execution time of the cell.

In [8]:
%%time
parameters = *initialParameters;
model.fitTo(*dataSet, RooFit::PrintLevel(-1),
            RooFit::NumCPU(2));
parameters.Print("V");

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss,bernstein)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss,bernstein)
RooMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (4.90896e+06) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	a1=0.8	a5=0.09	mean=10	sigFrac=0.1	sigma=-0.644826
PID612/RooRealMPFE::nll_model_modelData_7f90414a78a0_MPFE1[ arg=nll_model_modelData_GOF1 vars=(a1,a2,a3,a4,a5,mean,sigFrac,sigma) ]
     p.d.f value is Not-a-Number @ !refCoefNorm=(), !pdfs=(gauss = 0/-1.61634,bernstein = 0.173668/35.4), !coefficients=(sigFrac = 0.1)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(), !pdfs=(gauss = 0/-1.61634,bernstein = 0.17

### 2. Run same fit with fast batch evaluation
Using `BatchMode(true)`, we switch to the faster batch computation interface.

Compare the post-fit parameters with above, and see the difference in execution time.
The parameters are virtually identical, but the fit is *a lot* faster.

In [9]:
%%time
parameters = *initialParameters;
model.fitTo(*dataSet, RooFit::PrintLevel(-1),
            RooFit::NumCPU(2),
            RooFit::BatchMode(true));
parameters.Print("V");

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.
RooMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (4.90896e+06) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	a1=0.8	a5=0.09	mean=10	sigFrac=0.1	sigma=-0.644826
PID614/RooRealMPFE::nll_model_modelData_7f9040f673a0_MPFE1[ arg=nll_model_modelData_GOF1 vars=(a1,a2,a3,a4,a5,mean,sigFrac,sigma) ]
     p.d.f normalization integral is zero or negative.
	Int(gauss) = -1.616338 @ x=x=-25, mean=mean=10, sigma=sigma=-0.644826
     p.d.f normalization integral is zero or negative.
	Int(gauss) = -1.616338 @ x=x=-25, mean=mean=10, sigma=sigma=-0.644826

[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
  1) RooRealVar::      a1 = 0.802452 +/- 0.00394348
  2) RooRealVar::      a2 = 0
  3) RooRealVar::      a3 = 0.09
  4) RooRealVar:: 

### 3. Repeat above comparisons with only one CPU

In [10]:
%%time
parameters = *initialParameters;
model.fitTo(*dataSet, RooFit::PrintLevel(-1));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss,bernstein)
RooMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (4.90896e+06) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	a1=0.8	a5=0.09	mean=10	sigFrac=0.1	sigma=-0.644826
RooNLLVar::nll_model_modelData[ paramSet=(a1,a2,a3,a4,a5,mean,sigFrac,sigma) ]
     function value is NAN @ paramSet=(a1 = 0.8,a2 = 0,a3 = 0.09,a4 = 0.2,a5 = 0.09,mean = 10,sigFrac = 0.1,sigma = -0.644826)
RooGaussian::gauss[ x=x mean=mean sigma=sigma ]
     p.d.f normalization integral is zero or negative: -1.616338 @ x=x=-99.8885, mean=mean=10, sigma=sigma=-0.644826
     p.d.f normalization integral is zero or negative: -1.616338 @ x=x=-79.724, mean=mean=10, sigma=sigma=-0.644826
     p.d.f normalization integral is zero or negative: -1.616338 @ x=x=13.9811, mean=mea

In [11]:
%%time
parameters = *initialParameters;
model.fitTo(*dataSet, RooFit::PrintLevel(-1),
            RooFit::BatchMode(true));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
RooMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (4.90896e+06) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	a1=0.8	a5=0.09	mean=10	sigFrac=0.1	sigma=-0.644826
RooGaussian::gauss[ x=x mean=mean sigma=sigma ]
     p.d.f normalization integral is zero or negative.
	Int(gauss) = -1.616338 @ x=x=-25, mean=mean=10, sigma=sigma=-0.644826

[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
Time: 5.621582746505737 seconds.



## Plot post-fit model
Let's use jsroot to compare data and post-fit model.
Note that one should always plot the data first, so RooFit adjusts the normalisation of the PDF to match the data.

In [12]:
%jsroot on

TCanvas canv;
auto frame = x.frame();

dataSet->plotOn(frame);
model.plotOn(frame);
model.plotOn(frame, RooFit::Components("gauss"), RooFit::LineColor(kRed), RooFit::Name("signalOnly"));
frame->Draw();

// Let's build a legend.
// To do that, we first change the auto-generated title of the signalOnly curve:
dynamic_cast<RooCurve*>(frame->findObject("signalOnly"))->SetTitle("Signal Model");

auto leg = frame->BuildLegend();
leg->Draw();

canv.Draw();

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (gauss)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
